# CNN迁移学习

## 环境导入

In [1]:
import os
## 导入 Inceptionv3 模型
from keras.applications.inception_v3 import InceptionV3, preprocess_input

# 导入建立神经网络的基本模块
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

from keras.optimizers import *
from keras.losses import categorical_crossentropy

# 导入数据增强模块
import cv2
from keras_preprocessing.image import ImageDataGenerator

# 超参数调节
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

# 可视化
# from keras.utils import plot_model
# from keras_visualizer import visualizer
# from IPython.display import Image, SVG, display
import datetime

# 回调
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


2023-01-07 01:21:49.605600: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## 参数区

In [2]:
DATASET_PATH_ROOT = '/data/DataSets/TWITTER_IMG_SENT_2015/dataset/'
OUT_PATH_ROOT = '/data/Models/TWITTER_SENT_2015/'
OUT_LOG_PATH = OUT_PATH_ROOT + 'logs/'

TOTAL_EPOCH = 300

超参数

In [3]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([512, 1024]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.5, 1.0]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_L_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.0001]))
METRIC_ACCURACY = 'accuracy'

## 数据准备

In [4]:

# 训练集
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        # rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)

#验证集
val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        # rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)

# 测试集
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, )

# 数据输入
train_generator = train_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}train', target_size=(299, 299), batch_size=617)
val_generator = val_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}validation', target_size=(299, 299), batch_size=176)
test_generator = test_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}test', target_size=(299, 299), batch_size=89)

Found 617 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
Found 89 images belonging to 2 classes.


## 迁移学习

In [ ]:
# 输出日志
LOG_DIR = OUT_LOG_PATH + 'hparam_tuning/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '/'
os.environ['TENSORBOARD_BINARY'] = '/usr/local/miniconda3/envs/TensorFlow/bin/tensorboard'

# @formatter:off
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR} --port 6006 --bind_all
# @formatter:on

Launching TensorBoard...

In [ ]:
def start_transfer_learning(run_dir, hparams):
    # 构建基础模型
    base_model = InceptionV3(weights='imagenet', include_top=False)  #去掉最后一层

    # 增加新的输出层
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # 添加全局平均池化层 将 MxNxC 的张量转换成 1xC 张量，C是通道数
    x = Dense(hparams[HP_NUM_UNITS], activation='relu')(x)  # 添加一个全连接层
    x = Dropout(hparams[HP_DROPOUT])(x)  # 添加一个隐藏层
    predictions = Dense(2, activation='softmax')(x)  # 自定义自己的分类器，这是一个2分类的分类器
    model = Model(inputs=base_model.input, outputs=predictions)  # 构建我们需要训练的完整模型

    # 锁层
    for layer in base_model.layers:
        layer.trainable = False

    # 编译模型
    # if hparams[HP_OPTIMIZER] == "adam":
    #     optimizer = Adam(learning_rate=hparams[HP_L_RATE])
    # elif hparams[HP_OPTIMIZER] == "sgd":
    #     optimizer = SGD(learning_rate=hparams[HP_L_RATE])
    # else:
    #     raise ValueError("unexpected optimizer name")
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss='categorical_crossentropy', metrics=['accuracy'])  # rmsprop

    # 训练
    model.fit(train_generator,
              steps_per_epoch=1,  #800
              epochs=TOTAL_EPOCH,
              validation_data=val_generator,
              validation_steps=1,
              validation_freq=20,
              class_weight=None,  # 样本是均衡的
              callbacks=
              [
                      TensorBoard(log_dir=run_dir),  #可视化
                      hp.KerasCallback(run_dir, hparams),  #超参数
                      EarlyStopping(monitor='val_acc', patience=20),  # 早停选项
                      ReduceLROnPlateau(monitor='val_acc', patience=20),  # 学习率衰减
                      ModelCheckpoint(run_dir, monitor='val_acc', save_best_only=True, save_freq=20)  #检查点

              ],
              # workers=1,
              )
    # 在测试集上评估 profile_batch=5, histogram_freq=1
    # scores = model.evaluate_generator(test_generator)

    # return scores


## 超参数调整

In [ ]:
# 超参数运行
def run_hparams():
    session_num = 0
    for num_units in HP_NUM_UNITS.domain.values:
        for dropout_rate in HP_DROPOUT.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                # for learning_rate in HP_L_RATE.domain.values:
                start_transfer_learning(
                        LOG_DIR + f'run-{session_num}',
                        {
                                HP_NUM_UNITS: num_units,
                                HP_DROPOUT  : dropout_rate,
                                HP_OPTIMIZER: optimizer,
                                # HP_L_RATE   : learning_rate,
                        }
                )
                session_num += 1


run_hparams()